In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# D3.js
## Visualizing on the Web

This module should be useful in two ways:

1. It is a high-level overview of the D3 ecosystem.
2. It provides a template [taken from here](http://nbviewer.ipython.org/github/abarto/embedding_interactive_charts_on_an_ipython_notebook/blob/master/embedding_interactive_charts_on_an_ipython_notebook.ipynb#sub_est_2012_df_by_state_template) for how to embed HTML/JavaScript in an IPython notebook, and how to link up data you may analyze in pandas or other Python tooling with a web frontend, all from the same ipynb.*


\* *Beware*, with great power comes great responsibility, and separation of concerns is good engineering practice. 

In most cases, you probably want your analysis in one place (e.g. an IPython notebook); dump that data to a file, and whip up a separate `.html` file for your visualization code. 


## When to use D3
In general, D3 is not useful for "exploratory" visualization. In Python, you'll want to stick to things like `matplotlib` and pandas' nice built-in plotting. 

D3 *is*, as we've seen, useful for creating beautiful, interactive, and immediately clear visualizations that highlight exactly what you want the viewer to get out of the visualization. Instead of exploratory visualization power, D3 is a very useful "explanatory" visualization tool.

Without further ado, let's dive into a visualization example.

## Building interactive charts
First off, let's make some fancy-looking charts with cool interactive options. People love being able to click on things.

### Dataset and plan
This notebook's author loves Baltimore (his hometown) and public transit, so we'll be combining these two interests via [Baltimore's open datasets](https://data.baltimorecity.gov). In particular, we'll look at the [Charm City Circulator](https://en.wikipedia.org/wiki/Charm_City_Circulator), a free bus initiative for downtown Baltimore launched in 2010. 

- First, we'll take a look at the headways data, and after some basic aggregation we'll take a look at a plot of headways data vs. ridership numbers on a per-day basis.
- Then, using [NVD3](http://nvd3.org/), we'll look at some basic vital signs. How many people ride daily? How full is each bus? For a very rough comparison, Maryland's MTA averages ~400,000 riders/weekday as of 2012.
- We'll move on to breaking down ridership by weekday vs. weekend and line (unfortunately the released data does not get much more granular than this). Maybe we'll be able to spot useful trends, such as who is riding the bus (e.g. commuters vs. tourists - downtown buses like these could reasonably service both).


In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

#actually useful python things
from collections import OrderedDict
from json import dumps
from urllib2 import urlopen
import pandas as pd
from threading import Lock

In [ ]:
#do some data science

def get_year(date_obj):
  return date_obj.split("/")[-1]


#data described/found here
#https://data.baltimorecity.gov/browse?category=Transportation&limitTo=datasets&utf8=%E2%9C%93
headways = pd.read_csv(open("small_data/Charm_City_Circulator_Headways.csv"))
ridership = pd.read_csv(open("small_data/Charm_City_Circulator_Ridership.csv"))

ridership_averages_by_weekday = ridership[['day', 
                                       'orangeAverage', 
                                       'purpleAverage', 
                                       'greenAverage']]\
                                     .groupby('day', sort=True)\
                                     .mean() #get excited for lots of chaining in D3


ridership_averages_by_year = ridership[['orangeAverage',
                                        'purpleAverage',
                                        'greenAverage']]
ridership_averages_by_year['year'] = ridership['date'].apply(get_year)

ridership_averages_by_year = ridership_averages_by_year.groupby('year')\
                                                       .mean()
def timeToFloat(timeString):
  try:
    minutes, seconds = timeString.split(":")
  except:
    return None
  return float(minutes) + float(seconds)/60.

headways_averages = headways[['day', 'date', 'orangeHeadway', 'purpleHeadway', 'greenHeadway']]
ridership_averages = ridership[['day', 'date', 'orangeAverage', 'purpleAverage', 'greenAverage']]
merged = headways_averages.merge(ridership_averages)
merged.purpleHeadway = merged.purpleHeadway.apply(timeToFloat)
ridership_averages_by_year.to_json()

### D3 starts here
Cool, so we have some basic things to plot. Let's do some D3 now. 

In [ ]:
"""
 Surprise! one more python boilerplate thing.
 This is actually the magic that lets us communicate between python and JS.
 Brief bit of JS knowledge: 'window' is in the global namespace 
 (it represents this particular browser window),
 so we can store variables that we want to be able to access anywhere in our code.
"""
from IPython.display import Javascript
Javascript("""
           window.headwayVsRidership={};
           """.format(merged.to_json()))


In [ ]:
%%javascript
//load d3.js and nvd3
require.config({
    paths: {
        'nvd3': '//cdnjs.cloudflare.com/ajax/libs/nvd3/1.7.0/nv.d3.min',
        'd3': '//cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min',
        'nv': '//cdnjs.cloudflare.com/ajax/libs/nvd3/1.7.0/nv.d3.min'
    },
    shim: {
        nvd3: {
          exports: 'nv',
          deps: ['d3.global']
        }
    }
});

//pure boilerplate junk
//http://stackoverflow.com/questions/13157704/how-to-integrate-d3-with-require-js
define("d3.global", ["d3"], function(unused) {
  d3 = unused;
});


First, let's convert the data from pandas' strange JSON format into an array of objects, 1 per data point, with the expected attributes (what D3 [expects](https://github.com/mbostock/d3/wiki/Selections#data)):

In [ ]:
%%javascript
console.log("Open up the Chrome dev console and look for this text to investigate what pandas dumped for us.");
console.log(window.headwayVsRidership);
console.log("===================");

/*
 * As it turns out, pandas returns a dictionary with 
 * the top-level keys as the column headers, and values
 * as the list of values under that column.
 */
function convertPlotData(data){
  var convertedData = [];
  //pandas gives each column the same number of
  //objects, arbitrarily choose one to iterate over
  for(var i in data[Object.keys(data)[0]]){
    var convertedDatum = {};
    for(var key in data){
      convertedDatum[key] = data[key][i];
    }
    convertedData.push(convertedDatum);
  }
  return convertedData;
}

window.convertPlotData = convertPlotData;

Next, let's build a basic D3 scatter plot skeleton.

On the x-axis we'll plot the average headway that day, and on the y-axis we'll plot the average number of riders that day.

D3 works on 'method-chaining', much like pandas or jQuery, so each function call returns another object that we can operate on. For example, in the code below, when we call `d3.select(#chart1)`, the `select` function returns a D3-ified version of the JS object representing the `chart1` div we appended.

What makes D3 unique is the idea that _data_ is bound to _DOM Elements_ in a data binding step that we'll see below. This elegant abstraction means we can bind our data to any HTML elements we want and visualize them basically however we want to.

For our visualization, we'll use [SVG](http://www.w3schools.com/svg/), which allows for relatively easy drawing of elements on a 2-dimensional canvas.

In [ ]:
%%javascript
require(['d3'], function(d3){
  $("#chart1").remove();
  /*
   * TO NOTE: 'element' is defined by ipython,
   * and is the DOM element for the output cell.
   * Ipython also wraps it in JQuery (no need to include jquery in require.js).
   */
  element.append("<div id='chart1'></div>");
    
  //the first thing we want to do is set up our viewport
  $("#chart1").width("960px");
  $("#chart1").height("600px");   
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 900 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px") //ipython notebook hack
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g") //D3 returns us this 'g' element, it will be our main canvas
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")"); //axes will go in margins
    
  //now, we get our data
  var data = window.convertPlotData(window.headwayVsRidership);
    
  /*
   * These functions determine where 
   * within the canvas (as a 2D x,y coordinate system)
   * our dots will go. When we pass these into D3,
   * it iterates over data point dot and calls the function
   * passing in that data point object, so 'd' stands for datum.
   */
  var calcXPosition = function(d){
    if(d.purpleHeadway == null){
      return -100;
    }
    else{
      //headway values range from 10 - 22
      //we want to range from ~ 0 - 900
      return (d.purpleHeadway - 8) * (900 / (22 - 8) )
    }
  }
  var calcYPosition = function(d){
    if(d.purpleAverage == null){
      return -100;
    }
    else{
      //ridership values range from 0 - 8000
      //we want to range from ~ 0 - 500
      var height =  (d.purpleAverage) * (500 / 8000 );
      //remember, top left of SVG is (0,0), bottom right is ~(900, 500)
      return 500 - height;
    }
  }
  //this is the core 'D3' step
  svg.selectAll(".dot") //note this begins as empty selection
    .data(data) //data-binding step! this selection is now bound to the elements in our data array
    .enter() //new node for each datum not in selection
    .append("circle") //1 for each new node
    .attr("class", "dot")
    .attr("r", 3.5) //radius
    .attr("cx", calcXPosition) //x position
    .attr("cy", calcYPosition) //y position: position 0 is top, (500 - margin) is bottom

});

### So, what just happened?
Let's dive a little farther down the rabbit hole. 

If you examine the code, it becomes apparent that 

```javascript
 svg.selectAll(el)
   .data(data)
```

was the snippet of code that actually created the visualization. D3 works on a *data-binding* model, which means that you attach the data to an element (in particular `el` is an SVG window in this case), and D3 works its magic by creating an HTML object for every element in that dataset. 

- Step 1 in the binding is to select the HTML elements that will represent our data points
- Step 2 is to actually bind the data 
- Step 3, the functions after `data` is called, actually create, style, and position the elements that D3 will create based on the newly-bound data



Now, we want to prettify this basic plot with axes, make it a bit more interactive, and so on. Moreover, D3 has several convenience functions we can use so we don't have to, for example, make/scale our own axes or data points on the plot.

In [ ]:
%%javascript
require(['d3'], function(d3){
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
    
  //the first thing we want to do is set up our viewport
  $("#chart1").width("960px");
  $("#chart1").height("600px");   
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 900 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px") 
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g") 
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")"); 
    
  var data = window.convertPlotData(window.headwayVsRidership);

  //D3 likes passing functions around, making these useful
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  //rather than scale manually, we use d3.scale
  var xScale = d3.scale.linear().range([0, width]);
    
  //set the min/max values of our axes
  //the 1 is for a bit of padding
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);

  //d3 also has an axis convenience function, and 
  //it even lets us pass our scale in to generate ticks
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
    
  //this is the function formerly known as calcXPosition
  var xMap = function(d) { return xScale(xVals(d));};
      
  //same thing for y axis
  var yScale = d3.scale.linear().range([height, 0]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};

  //now, we add the axes in
    
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")") //slide it to bottom
      .call(xAxis) //this is a d3 idiom for SVG generation
    .append("text") //and we want to set a label
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
     .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)") //so text is vertical
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");
    
    

  svg.selectAll(".dot")
    .data(data)
    .enter()
    .append("circle") 
    .attr("class", "dot")
    .attr("r", 3.5)
    .attr("cx", xMap) 
    .attr("cy", yMap);

});

So this is a very basic, passable graph. However, it's not much better than something we could do in matplotlib. Let's add some interactivity, labeling, and color the dots to make differentiating weekdays/weekends easy.

In [ ]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
  var data = convertPlotData(window.headwayVsRidership);
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  var xScale = d3.scale.linear().range([0, width]);
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
  var xMap = function(d) { return xScale(xVals(d));};
      
  var yScale = d3.scale.linear().range([height, 0]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);

  
  //NEW: Coloring. We now color the graph, based on 
  //whether it's a weekend or weekday.
  //As with our other scaling, we determine the color
  //on a per-datum basis and let D3 do much of the heavy lifting.
  var cValue = function(d) { 
    if(d.day == "Saturday" || d.day == "Sunday"){
        return "Weekend";
    }
    return "Weekday";
  }
  var color = d3.scale.category10(); //we'll now call this during our graph construction
  
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");

  //NEW: TOOLTIP. 
  var tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
 
  svg.selectAll(".dot")
     .data(data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", xMap)
     .attr("cy", yMap)
     .style("fill", function(d) { return color(cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });
    
  //NEW: Legend
  var legend = svg.selectAll(".legend")
      .data(color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  // draw legend colored rectangles
  legend.append("rect")
      .attr("x", width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  // draw legend text
  legend.append("text")
      .attr("x", width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})

});

As it turns out, D3, while very extensible, takes a lot of work to use on its own. As good engineers, we're all about [looking for external libraries](https://www.google.com/webhp?&q=d3.js%20charts) when we want to do things better and more quickly. [NVD3](http://nvd3.org/examples/index.html) looks like a good candidate.

In [ ]:
Javascript("""
           window.averageByYear={}; 
           window.averageByWeekday={};
           """.format(ridership_averages_by_year.to_json(),
                      ridership_averages_by_weekday.to_json()))

In [ ]:
%%javascript
//again this function was created just by inspecting
//the data format in the example, no substantive knowledge of D3 required
function convertKeys(pandasJSON, keyOrder){

    var data = [];
    Object.keys(pandasJSON).forEach(function(key){
        var retObj = {};
        retObj['key'] = key;
        retObj['values'] = [];
        var color = key.substring(0, key.indexOf("A"));
        retObj['color'] = color;
        var order = Object.keys(pandasJSON[key]);
        if(keyOrder){
            order = keyOrder;
        }
        order.forEach(function(j){
            var innerObj = {};
            innerObj['x'] = j;
            innerObj['y'] = pandasJSON[key][j];
            retObj['values'].push(innerObj);
        });
        data.push(retObj);
    });
    return data;
}
window.convertKeys = convertKeys

In [ ]:
%%javascript
window.WEEKDAYS_IN_ORDER = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday"
]

require(['d3', 'nvd3'], function(d3, nv){
    //first we need to include nvd3's stylesheet because hacks
    $("head").append("<link rel='stylesheet' type='text/css' href='https://cdn.rawgit.com/novus/nvd3/v1.8.1/build/nv.d3.css' />");
    
    //generate data from pandas json into d3 format (done by copying the example)
    window.chart1data = window.convertKeys(window.averageByYear);
    window.chart2data = window.convertKeys(window.averageByWeekday, window.WEEKDAYS_IN_ORDER);
    
    //making some display ports
    element.append("<div id='chart1' />");
    element.append("<hr />");
    element.append("<div id='chart2' />");
    $("#chart1").append("<svg height='500px'></svg>");
    $("#chart2").append("<svg height='500px'></svg>");

    nv.addGraph(genGraph(window.chart1data, "#chart1 svg"));
    nv.addGraph(genGraph(window.chart2data, "#chart2 svg"));

});

function genGraph(data, el){
  return function() {
    var chart = nv.models.multiBarChart()
      .reduceXTicks(true)   //If 'false', every single x-axis tick label will be rendered.
      .rotateLabels(0)      //Angle to rotate x-axis labels.
      .showControls(true)   //Allow user to switch between 'Grouped' and 'Stacked' mode.
      .groupSpacing(0.1)    //Distance between each group of bars.
      .color(function(d, i){ //set color. fancy
          return d['color'];              
      })
    ;

    chart.yAxis
      .tickFormat(d3.format(',.0f'));

    d3.select(el)
      .datum(data)
      .call(chart);

    nv.utils.windowResize(chart.update);

    return chart;
  }
}


### Next steps and additional resources
Aside from just generally being prettier than the equivalent matplotlib/seaborn graph, this one is interactive - and who doesn't love interactive graphs. We've also answered a variety of the questions we set out to:

- Stacking the yearly graph shows us an upward trend in overall ridership, and side-by-side shows us that this trend holds true across all 3 lines.
- We can stack the 'riders by weekday' graph to see somewhere between 9-10,000 people ride on the average day. For a system with ~20 buses and 3 lines, this isn't bad at all.
- We can side-by-side the 'riders by weekday' graph to see that the purple line is the most popular. Within Baltimore's geography, this makes sense as that bus travels through some of the city's most populated areas.
- Each line receives its highest ridership on Friday and sees minor drops on the weekend - which may indicate the system is popular among both tourists and commuters, but primarily functions as a commuter bus.

And we determined this with just a few minutes of Googling and wrestling with D3! Notably, there's substantially less code if you _don't_ do this in IPython notebook.

A brief note on the data-binding: We used `datum` instead of `data` here. This is NVD3 magic, and is based on some predefined keys that are expected in each element of the data array (note the `x` and `y` conversion we did in `convertKeys`). 


Unfortunately, `nvd3` is only useful for certain, static charts. Never fear, though, because there exists a plethora of libraries that cover the boilerplate for a variety of charts, while still offering flexibility by being built on top of D3.

- [C3.js](http://c3js.org/)
- [Rickshaw](http://code.shutterstock.com/rickshaw/tutorial/introduction.html) - another popular time series library.
- [JSNetworkX](http://felix-kling.de/JSNetworkX/index.html) - NetworkX for JavaScript!
- [DataMaps](http://datamaps.github.io/) - for maps
- [Pyxley](https://github.com/stitchfix/pyxley/tree/master/examples/metricsgraphics) is a wrapper with strong Python/Pandas support.
- And finally, [a long list](http://mikemcdearmon.com/portfolio/techposts/charting-libraries-using-d3) for your perusal.

And of course, there are plenty of charting libraries outside of the D3 ecosystem, too.

## Live-updating visualizations
So remember how `enter` created new nodes only for data points that weren't already present? There is an analogous `exit` function that returns only formerly-bound nodes. We can use these to create live data visualizations where data enters and exits the visualization on the fly.  This is particularly useful for time series data and other streaming data. One time, this notebook's author wrote a [chat client](https://github.com/cmoscardi/bubblechat) that uses D3 to visualize a live chatroom via a tree of replies.

I think it's easy to imagine using this code to plot reddit conversations in real-time (or, for a big thread, animate the thread tree's growth over time).

In [ ]:
%%javascript
function buildTree(messages, activeMessage) {
  content.innerHTML = "";

  var margin = {top: 20, right: 120, bottom: 20, left: 120},
      width = 960 - margin.right - margin.left,
      height = 600 - margin.top - margin.bottom,
      r = 300;

  var tree = d3.layout.tree()
    .size([360, r]) //this is pretty fancy
    .separation(separation)
    .sort(function(a, b){
      return b.id - a.id;
    });


  var svg = d3.select("#content").append("svg")
    .attr("width", width + margin.right + margin.left)
    .attr("height", height + margin.top + margin.bottom)
    .style("float", "left")
    .append("g")
    .attr("transform", "translate(" + (r*1.5) + "," + (r) + ")");

  var messageArea = d3.select("#content").append("div")
    .style("float", "left")
    .style("width", "300px")
    .style("margin-top", "100px")
    .style('max-width', '300px')
    .attr('class', 'messageArea');


  messageArea
    .append('p')
    .attr('class', 'username')
    .attr('color', 'black');
  messageArea
    .append('p')
    .attr('class', 'message')
    .attr('color', 'black');


  var colors = d3.scale.category20();
  var messageColor = function(d){
    return d.username;
  }
  var source = messages[0];
  update(source);

  function update(root) {
    var nodes = tree.nodes(root);
    var links = tree.links(nodes);

    var link = svg.selectAll(".link")
      .data(links, function(d){return d.source.id + "-" + d.target.id});

    var linkEnter = link
      .enter()
      .append("line")
      .attr("class", "link")
      .attr("stroke", "black")
      .style("stroke-opacity", 1)
      .style("stroke-width", function(d) { return 1 })
      .attr("x1", function(d){return rtc(d.source, true).x})
      .attr("y1", function(d){return rtc(d.source, true).y})
      .attr("x2", function(d){return rtc(d.source, true).x})
      .attr("y2", function(d){return rtc(d.source, true).y})

    var node = svg.selectAll(".node")
      .data(nodes, function(d){return d.id})
      //keeps them above the links
      .each(function(){ this.parentNode.appendChild(this);});

    var nodeEnter = node
      .enter()
      .append("circle")
      .attr("class", "node")
      .attr("r", function(d) {return (50/(1 + d.depth))})
      .attr("fill", function(d){return colors(messageColor(d))})
      .attr("stroke", "black")
      .attr("transform", function(d) { return "rotate(" + (d.parent && d.parent.x0? d.parent.x0 - 90:0) + ")translate(" + (d.parent && d.parent.y0? d.parent.y0:0) + ")"})
      .on("mouseover", function(d){
        messageArea
          .style('background', colors(messageColor(d)));
        messageArea.select('p.username')
          .text("user: " + d.username);
        messageArea.select('p.message')
          .text("message: " + d.message);
      })
      .on("click", function(d){
        if(activeMessage.el){
          activeMessage.el.attr("stroke", "black");
        }
        activeMessage.message = d;
        activeMessage.el = $(this);
        $(this).attr("stroke", "red");
      });

    transitionLinks(link);
    transitionNodes(node);
    
    //set this for the transitions later
    nodes.forEach(function(n){
      n.x0 = n.x;
      n.y0 = n.y;
    });
  }
  return update;
}

function radialToCartesian(d, usex0){
  if(!d.x){
    d.x = 90;
  }
  if(!d.x0){
    d.x0 = 90;
  }
  var r = d.y;
  var theta = d.x;
  if(usex0){
    r = d.y0;
    theta = d.x0;
  }
  theta= (theta - 90) / 180 * Math.PI;
  return {x: r*Math.cos(theta), y: r*Math.sin(theta)};
}
var rtc = radialToCartesian;

function separation(a, b) {
  return (a.parent == b.parent ? .5 : 1) / a.depth;
}

function transitionLinks(link){
  link.transition()
      .duration(700)
      .attr("x1", function(d) { return rtc(d.source).x; })
      .attr("y1", function(d) { return rtc(d.source).y; })
      .attr("x2", function(d) { return rtc(d.target).x; })
      .attr("y2", function(d) { return rtc(d.target).y; });
}

function transitionNodes(node){
  node.transition()
      .duration(700)
      .attr("transform", function(d) { return "rotate(" + (d.x? d.x-90 :0) + ")translate(" + d.y + ")"}); 
}

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*